# Homework 01: Systematic state space traversal

In this homework we will explore state space traversals, to be precise we will imlement: random search, breadth-first search, depth-first search, heuristic greedy best-first search and A-star.

## Maze & auxillary classes

We will define two custom handy classes. One will be **ResultDict** class, it will represent result of path-finding algorithm. Another one will be **Maze** class, it will encapsulate file reading and present graph abstract node's neighborhood, **Maze** will also have **display** method to show path and maze itself with the following map legend:

    '█' - Wall cell
    'S' - Start of path
    'E' - End of path
    '+' - Cell included in the path
    '.' - Visited cell that was not included in a path
    ' ' - empty cell

In [1]:
from dataclasses import dataclass

Node = tuple[int, int]

@dataclass(frozen=True)
class ResultDictionary:
    """Class representing result of pathfinding algorithm"""
    distance : dict[Node, int]
    path : list[Node]

EMPTY_RESULT = ResultDictionary({}, [])

In [2]:
class Maze:

    def __init__(self, file_name:str, wall_char:str='X', empty_char:str=' '):
        self.maze = []
        self.height = 0
        self.width  =  0
        self.start  = None
        self.end    = None
        self._read_maze(file_name, wall_char, empty_char)

    def _read_maze(self, filename : str, wall_char:str, empty_char:str):
        with open(filename) as f:
            for line in f:
                match line[0]:
                    case 's':
                        self.start = eval('(' + line[6:] + ')') # LISP cursed
                    case 'e':
                        self.end = eval('(' + line[4:] + ')')
                    case _:
                        self.maze.append(list(map(lambda c: c == empty_char, line)))
        self.height = len(self.maze)
        self.width = len(self.maze[0])

    def neighbors(self, n:Node):
        for dx, dy in [(-1, 0), (0, -1), (1, 0), (0, 1)]:
            if self.maze[n[1] + dy][n[0] + dx]:
                yield (n[0] + dx, n[1] + dy)

    def display(self, result:ResultDictionary=EMPTY_RESULT):
        path_set = set(result.path)
        for i in range(self.height):
            for j in range(self.width):
                if not self.maze[i][j]:
                    print('█', end='')
                elif (j, i) == self.start:
                    print('S', end='')
                elif (j, i) == self.end:
                    print('E', end='')
                elif (j, i) in path_set:
                    print('+', end='')
                elif (j, i) in result.distance:
                    print('.', end='')
                else:
                    print(' ', end='')
            print('')
        if result.distance:
            print('Cells opened: ', len(result.distance))
            if result.path:
                print('Path length: ', len(result.path) - 1)
            else:
                print('End is unreacheable from start')

In [3]:
m36 = Maze('data/36.txt')
m36.display()

████████████████████████████████████████████████████████
█                                                 █   ██
█  ██ ████   █  █████ █████  █  ████   ████ █ █   █ █ ██
█ █                   █   █   █       █      S█ █ █ █ ██
█   ██ █  ██████ ████ █   ██  ███  █    ██ ███  █ ███ ██
█       █    E          █             █         █     ██
████████████████████████████████████████████████████████


## Implementing uninformed algorithms

Let's define a custom function to reconstruct path

In [4]:
def make_result(maze:Maze, distance:dict[Node, int], pred:dict[Node, Node]):
    path = [maze.end]
    while path[-1] != maze.start:
        path.append(pred[path[-1]])
    return ResultDictionary(distance, path[::-1])

### Random Search

We operate with array of open vertices, which will have starting cell at the beginning and distance map.
We will choose random element of open array, swap it with last and then pop from the list. Then we will open each of its neighbors and add them to open array. We will repeat this until exit is found during discovery fase or there is no new cells to visit i.e. we have exhausted our component.

In [5]:
from random import randint, seed

seed(42)

def RandomSearch(maze : Maze) -> ResultDictionary:
    distance = {maze.start:0}
    pred = {maze.start:maze.start}
    tovisit = [maze.start]
    if maze.start == maze.end:
        return ResultDictionary(distance, tovisit)
    
    while tovisit:
        idx = randint(0, len(tovisit) - 1)
        tovisit[idx], tovisit[-1] = tovisit[-1], tovisit[idx]
        current = tovisit.pop()

        for n in maze.neighbors(current):
            if n not in distance:
                tovisit.append(n)
                pred[n] = current
                distance[n] = distance[current] + 1
                if n == maze.end:
                    return make_result(maze,  distance, pred)
    return ResultDictionary(distance, [])
        

In [6]:
rs = RandomSearch(m36)
m36.display(rs)

████████████████████████████████████████████████████████
█.........+++++++++++++++++++++++++++.++++++......█...██
█..██.████+..█..█████.█████..█..████+++████+█.█...█.█.██
█.█......++...........█...█...█.......█....++S█.█.█.█.██
█...██.█.+██████.████.█...██..███..█....██.███..█.███.██
█..  ...█++++E .........█.............█.........█.....██
████████████████████████████████████████████████████████
Cells opened:  189
Path length:  48


### BFS

BFS is very similiar to Random Search, the only difference is that we will pop not random element, but the first pushed. In other words elements will be added and removed in "first in, first out" fashion.

In [7]:
from collections import deque

def BFS(maze : Maze) -> ResultDictionary:
    distance = {maze.start:0}
    pred = {maze.start:maze.start}
    tovisit = deque([maze.start])
    if maze.start == maze.end:
        return ResultDictionary(distance, tovisit)

    while tovisit:
        current = tovisit.popleft()

        for n in maze.neighbors(current):
            if n not in distance:
                tovisit.append(n)
                pred[n] = current
                distance[n] = distance[current] + 1
                if n == maze.end:
                    return make_result(maze,  distance, pred)
    return ResultDictionary(distance, [])

In [8]:
bfs = BFS(m36)
m36.display(bfs)

████████████████████████████████████████████████████████
█           ......................................█...██
█  ██ ████   █..█████.█████..█..████...████.█.█...█.█.██
█ █           ........█...█...█.......█++++++S█.█.█.█.██
█   ██ █  ██████.████.█+++██..███..█++++██.███..█.███.██
█       █    E++++++++++█++++++++++++.█.........█.....██
████████████████████████████████████████████████████████
Cells opened:  146
Path length:  36


### DFS

We will implement iterative DFS with stack. On every iteration we will look at stack top's neighbors, if at least one of its neighbors is not visited, it will be pushed into stack and iterations will conitinue, otherwise stack will be popped.

In [9]:
def DFS(maze:Maze) -> ResultDictionary:
    distance = {maze.start:0}
    pred = {maze.start:maze.start}
    stack = [maze.start]
    if maze.start == maze.end:
        return ResultDictionary(distance, stack)
        
    while stack:
        current = stack[-1]
        do_pop = True

        for n in maze.neighbors(current):
            if n not in distance:
                do_pop = False
                stack.append(n)
                pred[n] = current
                distance[n] = distance[current] + 1
                if n == maze.end:
                    return make_result(maze,  distance, pred)
        if do_pop:
            stack.pop()
    return ResultDictionary(distance, [])

In [10]:
dfs = DFS(m36)
m36.display(dfs)

████████████████████████████████████████████████████████
█             .+++++++......++++......++++++.+++. █   ██
█  ██ ████   █.+█████+█████.+█.+████.++████+█+█+. █ █ ██
█ █           .++.  .+█+++█.+.█++++..+█  .++.S█+█ █ █ ██
█   ██ █  ██████+████+█+.+██+.███.+█.+. ██+███.+█ ███ ██
█       █    E+++....+++█++++.....++++█  .++++++█     ██
████████████████████████████████████████████████████████
Cells opened:  120
Path length:  74


## Implementing informed algorithms

We will create utility dataclass **PrioritizedNode** to use it in heap. Additionally we will define three heuristics coresponding to $l_1$, $l_2$ and $l_{+\infty}$ distance to maze exit.

In [11]:
from heapq import heappush, heappop
from dataclasses import field
from math import sqrt

@dataclass(order=True)
class PrioritizedNode:
    node: Node=field(compare=False)
    priority: float|int = 0.0
    distance:int=field(compare=False, default=0)
    
    def __iter__(self):
        return iter((self.node, self.distance))

def L1_dist(maze : Maze):
    def l1(n : Node):
        return abs(n[0] - maze.end[0]) + abs(n[1] - maze.end[1])
    return l1

def L2_dist(maze : Maze):
    def l2(n : Node):
        return sqrt((n[0] - maze.end[0])**2 + (n[1] - maze.end[1])**2)
    return l2

def Chebyshev_dist(maze : Maze):
    def chebyshev(n : Node):
        return max((n[0] - maze.end[0]), abs(n[1] - maze.end[1]))
    return chebyshev

### Greedy best-first search

Our implementation of Greedy Search will use heap to keep track of a cell closest to end.

In [12]:
def GreedySearch(maze:Maze, heuristic) -> ResultDictionary:
    distance = {maze.start:0}
    pred = {maze.start:maze.start}
    heap = [PrioritizedNode(maze.start)]
    if maze.start == maze.end:
        return ResultDictionary(distance, heap)

    while heap:
        current = heappop(heap).node
        for n in maze.neighbors(current):
            if n not in distance:
                pred[n] = current
                distance[n] = distance[current] + 1
                heappush(heap, PrioritizedNode(n, priority=heuristic(n)))
                if n == maze.end:
                    return make_result(maze,  distance, pred)
    return ResultDictionary(distance, [])

In [13]:
gs = GreedySearch(m36, Chebyshev_dist(m36))
m36.display(gs)

████████████████████████████████████████████████████████
█             .++++++++++++++++.                  █   ██
█  ██ ████...█.+█████.█████..█++████.  ████.█.█   █ █ ██
█ █     .+++++++.     █   █   █++++++.█++++++S█ █ █ █ ██
█   ██ █.+██████ ████ █   ██  ███..█++++██.███  █ ███ ██
█       █++++E          █           ..█.        █     ██
████████████████████████████████████████████████████████
Cells opened:  71
Path length:  48


### A*

A-star's implementation is very similiar to one of Greedy search. Differences are that our priority will be a sum of heuristic and length of path to opened node and introduction of relaxation. We will not use decrease key, but rather skip already node if we try to process it, when it was already closed.

In [14]:
def Astar(maze:Maze, heuristic) -> ResultDictionary:
    distance = {maze.start:0}
    pred = {maze.start:maze.start}
    heap = [PrioritizedNode(maze.start)]
    if maze.start == maze.end:
        return ResultDictionary(distance, heap)
        
    while heap:
        current, current_distance = heappop(heap)
        if current_distance > distance[current]:
            continue
        for n in maze.neighbors(current):
            if n not in distance or distance[n] > distance[current] + 1:
                pred[n] = current
                distance[n] = distance[current] + 1
                heappush(heap, PrioritizedNode(n, distance=distance[n], priority=heuristic(n)+distance[n]))
                if n == maze.end:
                    return make_result(maze,  distance, pred)
    return ResultDictionary(distance, [])

In [15]:
astar = Astar(m36, L1_dist(m36))
m36.display(astar)

████████████████████████████████████████████████████████
█                                          . .    █   ██
█  ██ ████   █  █████ █████  █ .████.. ████.█.█   █ █ ██
█ █                   █...█ ..█.......█++++++S█ █ █ █ ██
█   ██ █  ██████.████.█+++██..███..█++++██.███  █ ███ ██
█       █    E++++++++++█++++++++++++.█......   █     ██
████████████████████████████████████████████████████████
Cells opened:  70
Path length:  36


## Heuristic analysis

This is part is not obligatory, but I thought it would be interesting to investigate this topic ($S$ is space of maze cells).

For further discussion let us define two important heuristics, for all $n \in S$:

$$
    h^0(n) = 0 = \text{trivial heuristic}
$$

$$
    h^*(n) = \text{length of the shortest path from $n$ to maze's end} = \text{optimal heuristic}
$$

And define $h_1, h_2, h_\infty$ to be $l_1$, $l_2$ and $l_\infty$ heurstics respectivly. Then for all $n \in S$ one can easily show following.

$$
    h^0(n) \leq h_\infty(n) \leq h_2(n) \leq h_1(n) \leq h^*(n)
$$

All those heuristics are then admissible. One can expect that the closer heuristic is to perfect heuristic, the less cells will open, because this heuristic are more "informative".

Let's implement $h_0$ and $h^*$.

In [16]:
def TrivialHeuristic(*_):
    return lambda x: 0

def OptimalHeuristic(maze : Maze):
    distance = {maze.end:0}
    tovisit = deque([maze.end])
    
    while tovisit:
        current = tovisit.popleft()
        for n in maze.neighbors(current):
            if n not in distance:
                tovisit.append(n)
                distance[n] = distance[current] + 1
    return lambda n : distance[n]

In [17]:
gs_star = GreedySearch(m36, OptimalHeuristic(m36))
m36.display(gs_star)

████████████████████████████████████████████████████████
█                                                 █   ██
█  ██ ████   █  █████ █████  █  ████   ████.█.█   █ █ ██
█ █                   █...█   █     ..█++++++S█ █ █ █ ██
█   ██ █  ██████.████.█+++██..███..█++++██.███  █ ███ ██
█       █    E++++++++++█++++++++++++.█.        █     ██
████████████████████████████████████████████████████████
Cells opened:  53
Path length:  36


In [18]:
dijkstra = Astar(m36, TrivialHeuristic(m36))
m36.display(dijkstra)

████████████████████████████████████████████████████████
█           ......................................█...██
█  ██ ████   █..█████.█████..█..████...████.█.█...█.█.██
█ █          .........█...█...█.......█++++++S█.█.█.█.██
█   ██ █  ██████.████.█+++██..███..█++++██.███..█.███.██
█       █    E++++++++++█++++++++++++.█.........█.....██
████████████████████████████████████████████████████████
Cells opened:  147
Path length:  36


## Display functions & data

In [19]:
import time

def display_uninformed(maze : Maze):
    for algo, name in zip([RandomSearch, BFS, DFS], ['Random Search:', 'BFS:          ', 'DFS:          ']):
        start = time.time()
        res = algo(maze)
        end = time.time()
        print(f"{name} Opened: {len(res.distance):<7}  Path: {len(res.path) - 1:<4}  Time: {1000*(end-start):.2f}ms")

def display_informed(maze : Maze):
    for heur, name in zip([Chebyshev_dist, L2_dist, L1_dist], ['L_inf:', 'L2:   ', 'L1:   ']):
        start = time.time()
        res = GreedySearch(maze, heur(maze))
        end = time.time()
        print(f"Greedy Search with {name} Opened: {len(res.distance):<7}  Path: {len(res.path) - 1:<4}  Time: {1000*(end-start):.2f}ms")
    
    print('')
    for heur, name in zip([Chebyshev_dist, L2_dist, L1_dist], ['L_inf:', 'L2:   ', 'L1:   ']):
        start = time.time()
        res = Astar(maze, heur(maze))
        end = time.time()
        print(f"A* with {name}            Opened: {len(res.distance):<7}  Path: {len(res.path) - 1:<4}  Time: {1000*(end-start):.2f}ms")
    
    print('')
    for heur, name in zip([TrivialHeuristic, OptimalHeuristic], ['trivial heuristic:', 'optimal heuristic:']):
        start = time.time()
        res = GreedySearch(maze, heur(maze))
        end = time.time()
        print(f"Greedy Search with {name} Opened: {len(res.distance):<7}  Path: {len(res.path) - 1:<4}  Time: {1000*(end-start):.2f}ms")
    print('')
    for heur, name in zip([TrivialHeuristic, OptimalHeuristic], ['trivial heuristic:', 'optimal heuristic:']):
        start = time.time()
        res = Astar(maze, heur(maze))
        end = time.time()
        print(f"A* with {name}            Opened: {len(res.distance):<7}  Path: {len(res.path) - 1:<4}  Time: {1000*(end-start):.2f}ms")

def display_algos(maze : Maze):
    display_uninformed(maze)
    print('')
    display_informed(maze)

display_algos(m36)

Random Search: Opened: 153      Path: 42    Time: 1.23ms
BFS:           Opened: 146      Path: 36    Time: 0.90ms
DFS:           Opened: 120      Path: 74    Time: 0.60ms

Greedy Search with L_inf: Opened: 71       Path: 48    Time: 1.04ms
Greedy Search with L2:    Opened: 53       Path: 36    Time: 0.71ms
Greedy Search with L1:    Opened: 53       Path: 36    Time: 0.68ms

A* with L_inf:            Opened: 118      Path: 36    Time: 2.01ms
A* with L2:               Opened: 107      Path: 36    Time: 1.80ms
A* with L1:               Opened: 70       Path: 36    Time: 1.15ms

Greedy Search with trivial heuristic: Opened: 150      Path: 38    Time: 1.56ms
Greedy Search with optimal heuristic: Opened: 53       Path: 36    Time: 1.44ms

A* with trivial heuristic:            Opened: 147      Path: 36    Time: 4.51ms
A* with optimal heuristic:            Opened: 53       Path: 36    Time: 1.80ms


Let's check our algorithms on the largest maze we have.

In [20]:
m332 = Maze('data/332.txt')
display_algos(m332)

Random Search: Opened: 99824    Path: 374   Time: 401.46ms
BFS:           Opened: 103733   Path: 332   Time: 241.03ms
DFS:           Opened: 1380752  Path: 9664  Time: 5365.02ms

Greedy Search with L_inf: Opened: 1850     Path: 538   Time: 12.30ms
Greedy Search with L2:    Opened: 646      Path: 386   Time: 4.34ms
Greedy Search with L1:    Opened: 687      Path: 392   Time: 4.11ms

A* with L_inf:            Opened: 45199    Path: 332   Time: 403.80ms
A* with L2:               Opened: 21602    Path: 332   Time: 197.06ms
A* with L1:               Opened: 2337     Path: 332   Time: 19.48ms

Greedy Search with trivial heuristic: Opened: 26037    Path: 1746  Time: 155.93ms
Greedy Search with optimal heuristic: Opened: 558      Path: 332   Time: 3238.28ms

A* with trivial heuristic:            Opened: 103550   Path: 332   Time: 868.42ms
A* with optimal heuristic:            Opened: 1343     Path: 332   Time: 3110.88ms


As expected only __A*__, **BFS** and **Greedy search with optimal heuristic** found correct solution. The least opened cells were opend by __A*__ with optimal heuristic, but due to its computational inefficency total time was much higher. Optimal for our use case is __A*__ with $l_1$ based heuristic, which compromises heuristic accuracy and speed.

## Tests

In [21]:
cumulative_time = 0
for i, d in enumerate([0, 4, 6, 26, 36, 42, 72, 84, 114, 220, 332]):
    maze = Maze('data/' + str(d) + '.txt')
    start = time.time()
    res = Astar(maze, L1_dist(maze))
    end = time.time()
    if len(res.path) - 1 == d:
        print(f'Passed test #{i:0>2}: time elapsed - {1000*(end - start):.2f}ms')
    else:
        print(f'Failed test #{i:0>2}')
    cumulative_time += 1000*(end - start)
print(f'\n    Total time {cumulative_time:.2f}ms')

Passed test #00: time elapsed - 0.01ms
Passed test #01: time elapsed - 0.05ms
Passed test #02: time elapsed - 0.06ms
Passed test #03: time elapsed - 0.60ms
Passed test #04: time elapsed - 0.33ms
Passed test #05: time elapsed - 0.17ms
Passed test #06: time elapsed - 1.87ms
Passed test #07: time elapsed - 5.64ms
Passed test #08: time elapsed - 8.18ms
Passed test #09: time elapsed - 18.95ms
Passed test #10: time elapsed - 18.69ms

    Total time 54.55ms


In [22]:
m = Maze('data/26.txt')
m.display(Astar(m, L1_dist(m)))

██████████████████████████████████
█         █   █                 ██
█ ███ █   █ █   █████ █   ███ █ ██
█     █ █   █ █   █   █ █   █   ██
█  ████   ███ ███ █  ███ █  █   ██
█     █         █ █         █   ██
█  ██  ██ ██ ██ █ █ █.███.████  ██
█   █     █       █ ........... ██
█ █  ██     █ ███ █████.█.█.█.█.██
█   █     █      .......█...█S..██
██  █ █ ██ █ ███████.██..███.+█.██
█   █               █ █.█...█+█.██
█  █ ███████ ██   █ █ █.█.█.█+..██
█               █ █   ....█..+█.██
████ ████ █ ███.█. █. ███████+█.██
█         █  E++++█...█+++++++..██
█  ███  █ █ █  .█+██.██+..█..█..██
█ █     █ █   █ █+++++++█...... ██
█  ██   █ ███   ██.█.█.████.█ █ ██
█     █ █     █             █   ██
█ █ █  ███████  ████████ █ ██ █ ██
█   █                       █   ██
██   █████ █  █ █ █ █ █████  ██ ██
█   █             █           █ ██
█ █   ██ ██ ███ █ █ █   ██    █ ██
█   █       █             █     ██
█    ██ ███ █  ████ ███   ██ █████
█ █     █   █   █               ██
█ ████  █    █  █ █ 